# Density-Based Spatial Clustering of Applications with Noise (DBSCAN)

This repository contains the source code to an algorithm that utilizes an unsupervised machine learning algorithm to separate Berkeley Seismology Laboratory's Power Density Function graphs into groups with similar distributions. This is particularly useful in deriving the training dataset for a neural network, since most stations contain a majority of "correct" measurements, allowing the DBSCAN's outlier detection to excel at extracting graphs that should be labeled "erroneous". 

In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import DBSCAN
from tqdm import tqdm
import skimage

Set the current working directory to a folder containing all the data.

In [ ]:
os.chdir("/home/gcl/TT/sylvesterseo/bsl/preserved_data")

The following code is used to visualize a PDF file, which is composed of 122 by 151 entries of a probability entry.

In [ ]:
def plotPDF(path):
    data = np.fromfile(path, sep=" ")

    # Plot Data
    data = np.reshape(data, (18422, 3))
    data = np.swapaxes(data, 0, 1)
    x = data[0].reshape((122, 151))
    y = data[1].reshape((122, 151))
    z = data[2].reshape((122, 151))

    fig, ax = plt.subplots()

    rainbow = matplotlib.colors.LinearSegmentedColormap.from_list(name="rainbow", colors=['white', 'magenta', 'blue', 'cyan', 'lime', 'yellow', 'orange', 'red'])
    c = ax.pcolormesh(x, y, z, cmap=rainbow, vmin=0, vmax=0.30)
    ax.set_title(path)
    ax.axis([-1.69897, 2.854109, -200, -50])
    fig.colorbar(c, ax=ax)

    plt.plot()

The following code is used to visualize multiple PDF files at once, taking the argument of a 2-dimensional array composed of file paths.

In [ ]:
def plotGroupPDF(paths, rows=5, cols=5):
    fig, ax = plt.subplots(nrows=rows, ncols=cols)
    rainbow = matplotlib.colors.LinearSegmentedColormap.from_list(name="rainbow", colors=['white', 'magenta', 'blue', 'cyan', 'lime', 'yellow', 'orange', 'red'])

    for col in range(cols):
        for row in range(rows):
            if row == 0:
                ax[row][col].set_title(f"Group {col - 1}")

            # Global settings for all subplots
            ax[row][col].set_xticklabels([])
            ax[row][col].set_yticklabels([])
            ax[row][col].tick_params(left=False, bottom= False)

            if col >= len(paths) or row >= len(paths[col]):
                continue

            path = paths[col][row]
            data = np.fromfile(path, sep=" ")

            data = np.reshape(data, (18422, 3))
            data = np.swapaxes(data, 0, 1)
            x = data[0].reshape((122, 151))
            y = data[1].reshape((122, 151))
            z = data[2].reshape((122, 151))

            c = ax[row][col].pcolormesh(x, y, z, cmap=rainbow, vmin=0, vmax=0.30)
            ax[row][col].axis([-1.69897, 2.854109, -200, -50])
            # fig.colorbar(c, ax=ax)

The DBSCAN algorithm has been tested to work best at around 1000 datasets.

In [ ]:
starting_index = 0
num_data = 1000

The following code stores the datapoints of each PDF file into a Python dictionary so that the data can be easily retrieved while the DBSCAN algorithm runs. It reads all the files that are inside the working directory (which was set earlier), starting from the file located at the `starting_index` position and storing `num_data` number of datapoints.

In [ ]:
file_list = os.listdir()
zMap = {}
for i in tqdm(range(num_data)):
    arr = np.fromfile(file_list[starting_index + i], sep=" ")[2::3]
    arr = arr.reshape(151, 122)
    arr = skimage.measure.block_reduce(arr, (2, 2), np.mean).flatten()
    zMap[i] = arr

In [ ]:
def metric(pathA, pathB):
    return np.sum(np.abs(np.subtract(zMap[int(pathA[0])], zMap[int(pathB[0])])))

This cell is responsible for actually conducting the DBSCAN algorithm. For documentation on scikit-learn's DBSCAN algorithm, please refer to [this link](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). For 1000 datasets, an epsilon value between 23 to 26 has been tested to be the most ideal.

In [ ]:
global string_seqs
string_seqs = np.arange(0, num_data).reshape(-1, 1)
clustered_dataset = DBSCAN(eps = 20, metric=metric, n_jobs=-1).fit(X=string_seqs)

In [ ]:
min_group = min(clustered_dataset.labels_)
max_group = max(clustered_dataset.labels_)
print(f"There are {max_group - min_group + 1} groups present, ranging from Group {min_group} to Group {max_group}.")

In [ ]:
total_count = {}
label_categories = {}
for i in range(len(clustered_dataset.labels_)):
    label = clustered_dataset.labels_[i]

    if label not in total_count:
        total_count[label] = 0
    if label not in label_categories:
        label_categories[label] = []
        
    total_count[label] += 1
    label_categories[label].append(i)

groups = []

for key in sorted(total_count.keys()):
    print(f"Group {key}: {total_count[key]} found")

    curr_group = []
    for i in range(min(5, total_count[key])):
        curr_group.append(file_list[label_categories[key][i]])
    
    groups.append(curr_group)

plotGroupPDF(groups, 5, 5)
        